In [1]:
import wandb
import yaml

In [2]:
api = wandb.Api()

In [3]:
training_config_filepath = "../experiments/training-era5-main/base-config.yaml"
with open(training_config_filepath, "r") as f:
    training_config = yaml.safe_load(f)
    out_names = training_config["stepper"]["out_names"]

In [4]:
# these IDs are for inference runs that twelve ICs each 5 years long, spanning 1940-2000
ids = {
    'rs0-era5': 'z8rm24sl',
    'rs1-era5': '1slxcmd9',
    'rs2-era5': 'zd8uq41z',
    'rs3-era5': 'afkfq8d7',
    'rs0-shield': 'vvv1zioj',
    'rs1-shield': 'n0fzhbm8',
    'rs2-shield': 'nyne4vm0',
    'rs3-shield': 'fpwivzys',
    'no-mois-cons-RS0': 'sldd1zin',
    'no-mois-cons-RS1': 'oqpxipfy',
    'no-mois-dryair-cons-RS0': 'd6h8c8du',
    'no-mois-dryair-cons-RS1': '45j0sg10',
}

In [5]:
metrics = {}
for name, id in ids.items():
    run = api.run(f"ai2cm/ace/{id}")
    metrics[name] = {}
    for out_name in out_names:
        metrics[name][out_name] = run.summary['inference/time_mean_norm/rmse/' + out_name]

In [6]:
def computed_weighted_mean(channel_rmses, weights):
    running_sum = 0
    running_denominator_sum = 0
    for name, value in channel_rmses.items():
        weight = weights.get(name, 1)
        running_sum += value * weight
        running_denominator_sum += weight
    return running_sum / running_denominator_sum

In [7]:
weights = {}
weighted_channel_mean = {}
for name in ids:
    weighted_channel_mean[name] = computed_weighted_mean(metrics[name], weights)
weighted_channel_mean

{'rs0-era5': 0.05397498693317175,
 'rs1-era5': 0.034195146271958944,
 'rs2-era5': 0.038642997406423095,
 'rs3-era5': 0.044930524732917546,
 'rs0-shield': 0.0519686390273273,
 'rs1-shield': 0.02463358772918582,
 'rs2-shield': 0.024683880195952954,
 'rs3-shield': 0.0393001629319042,
 'no-mois-cons-RS0': 0.04444670990575105,
 'no-mois-cons-RS1': 0.02279729621950537,
 'no-mois-dryair-cons-RS0': 0.033312641782686114,
 'no-mois-dryair-cons-RS1': 0.02624955619685352}

In [8]:
weights = {'specific_total_water_0': 0.1}
weighted_channel_mean = {}
for name in ids:
    weighted_channel_mean[name] = computed_weighted_mean(metrics[name], weights)
weighted_channel_mean

{'rs0-era5': 0.044863832116794684,
 'rs1-era5': 0.03205665309634628,
 'rs2-era5': 0.029033424677953217,
 'rs3-era5': 0.031185155271603718,
 'rs0-shield': 0.03946695795051313,
 'rs1-shield': 0.020915887639108597,
 'rs2-shield': 0.01817166385034103,
 'rs3-shield': 0.027587600027698486,
 'no-mois-cons-RS0': 0.023019419460464458,
 'no-mois-cons-RS1': 0.018357124984787886,
 'no-mois-dryair-cons-RS0': 0.025761329030234918,
 'no-mois-dryair-cons-RS1': 0.018758003266107163}